# Анализ эффективности удержания
В этом задании вам предлагается проанализировать данные одной из американских телекоммуникационных компаний  о пользователях, которые потенциально могут уйти.

Измерены следующие признаки:

* state — штат США;
* account_length — длительность использования аккаунта;
* area_code — деление пользователей на псевдорегионы, использующееся в телекоме;
* intl_plan — подключена ли у пользователя услуга международного общения;
* vmail_plan — подключена ли у пользователя услуга голосовых сообщений;
* vmail_message — количество голосых сообщений, который пользователь отправил / принял;
* day_calls — сколько пользователь совершил дневных звонков;
* day_mins — сколько пользователь проговорил минут в течение дня;
* day_charge — сколько пользователь заплатил за свою дневную активность;
* eve_calls, eve_mins, eve_charge — аналогичные метрики относительно вечерней активности;
* night_calls, night_mins, night_charge — аналогичные метрики относительно ночной активности;
* intl_calls, intl_mins, intl_charge — аналогичные метрики относительно международного общения;
* custserv_calls — сколько раз пользователь позвонил в службу поддержки;
* treatment — номер стратегии, которая применялись для удержания абонентов (0, 2 = два разных типа воздействия, 1 = контрольная группа);
* mes_estim — оценка интенсивности пользования интернет мессенджерами;
* churn — результат оттока: перестал ли абонент пользоваться услугами оператора.

Давайте рассмотрим всех пользователей из контрольной группы (treatment = 1). Для таких пользователей мы хотим проверить гипотезу о том, что штат абонента не влияет на то, перестанет ли абонент пользоваться услугами оператора. 

Для этого мы воспользуемся критерием хи-квадрат.  Постройте таблицы сопряженности между каждой из всех 1275 возможных неупорядоченных пар штатов и значением признака churn.  Для каждой такой таблицы 2x2 применить критерий хи-квадрат можно с помощью функции:

scipy.stats.chi2_contingency(subtable, correction=False)

Заметьте, что, например, (AZ, HI)  и  (HI, AZ) — это одна и та же пара. Обязательно выставьте correction=False (о том, что это значит, вы узнаете из следующих вопросов).

Сколько  достигаемых уровней значимости оказались меньше, чем $α=0.05$?

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from itertools import combinations

In [2]:
df = pd.read_csv('churn_analysis.csv')

In [3]:
df.head(10)

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,1,OH,107,415,no,yes,26,161.6,123,27.47,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,2,NJ,137,415,no,no,0,243.4,114,41.38,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,3,OH,84,408,yes,no,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,4,OK,75,415,yes,no,0,166.7,113,28.34,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.
5,5,AL,118,510,yes,no,0,223.4,98,37.98,...,203.9,118,9.18,6.3,6,1.70,0,2,0.60,False.
6,6,MA,121,510,no,yes,24,218.2,88,37.09,...,212.6,118,9.57,7.5,7,2.03,3,0,0.28,False.
7,7,MO,147,415,yes,no,0,157.0,79,26.69,...,211.8,96,9.53,7.1,6,1.92,0,0,0.59,False.
8,8,LA,117,408,no,no,0,184.5,97,31.37,...,215.8,90,9.71,8.7,4,2.35,1,1,0.50,False.
9,9,WV,141,415,yes,yes,37,258.6,84,43.96,...,326.4,97,14.69,11.2,5,3.02,0,2,0.45,False.


In [4]:
df_treatment_1 = df[df['treatment'] == 1]
df_treatment_1['churn'][df_treatment_1['churn'] == 'False.'] = 0
df_treatment_1['churn'][df_treatment_1['churn'] == 'True.'] = 1

/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/pandas/core/generic.py:8682: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2878: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [5]:
df_treatment_1.head(10)

,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,0
3,3,OH,84,408,yes,no,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,0
8,8,LA,117,408,no,no,0,184.5,97,31.37,...,215.8,90,9.71,8.7,4,2.35,1,1,0.50,0
12,12,IA,168,408,no,no,0,128.8,96,21.90,...,141.1,128,6.35,11.2,2,3.02,1,1,0.37,0
17,17,VT,93,510,no,no,0,190.7,114,32.42,...,129.6,121,5.83,8.1,3,2.19,3,1,0.84,0
21,21,CO,77,408,no,no,0,62.4,89,10.61,...,209.6,64,9.43,5.7,6,1.54,5,1,0.47,1
26,26,WY,57,408,no,yes,39,213.0,115,36.21,...,182.7,115,8.22,9.5,3,2.57,0,1,0.63,0
28,28,MO,20,415,no,no,0,190.0,109,32.30,...,181.5,102,8.17,6.3,6,1.70,0,1,0.78,0
32,32,LA,172,408,no,no,0,212.0,121,36.04,...,293.3,78,13.20,12.6,10,3.40,3,1,0.51,0
35,35,GA,72,415,no,yes,37,220.0,80,37.40,...,152.8,71,6.88,14.7,6,3.97,3,1,0.41,0


In [6]:
states_list = list(set(df_treatment_1['state']))
states_treatment_1_churn_counts_lib = {}
for state in states_list:
    df_treatment_1_current_state = df_treatment_1[df_treatment_1['state'] == state]
    state_churn_0 = len(df_treatment_1_current_state[df_treatment_1_current_state['churn'] == 0])
    state_churn_1 = len(df_treatment_1_current_state[df_treatment_1_current_state['churn'] == 1])
    states_treatment_1_churn_counts_lib[state] = [state_churn_0, state_churn_1]
states_treatment_1_churn_counts_pd = pd.DataFrame(states_treatment_1_churn_counts_lib).T
states_treatment_1_churn_counts_pd.head()

,0,1
AK,19,1
AL,25,5
AR,11,5
AZ,17,2
CA,10,5


In [7]:
states_comb = list(combinations(states_list, 2))

In [8]:
contingency_table_list = []
for states_pair in states_comb:
    contingency_table = []
    contingency_table.append(states_treatment_1_churn_counts_lib[states_pair[0]])
    contingency_table.append(states_treatment_1_churn_counts_lib[states_pair[1]])
    contingency_table_list.append(pd.DataFrame(np.array(contingency_table),
                                               index=[states_pair[0], states_pair[1]]))

In [9]:
chi2_pvalue_lower_0_05_count = 0
for contingency_table in contingency_table_list:
    if stats.chi2_contingency(contingency_table, correction=False)[1] < 0.05:
        chi2_pvalue_lower_0_05_count += 1
print 'Количество пар, в которых p-value меньше 0.05: ', chi2_pvalue_lower_0_05_count

Количество пар, в которых p-value меньше 0.05:  34


В основе критерия xи-квадрат лежит предположение о том, что если верна нулевая гипотеза, то <b><i>дискретное</i></b> биномиальное распределение данных по клеткам в  таблице сопряженности  может быть аппроксимировано с помощью <b><i>непрерывного</i></b> распределения xи-квадрат.  Однако точность такой аппроксимации существенно зависит от суммарного количества наблюдений и их распределения в этой таблице (отсюда и ограничения при использовании критерия xи-квадрат).

Одним из способов коррекции точности аппроксимации является поправка Йетса на непрерывность.  Эта поправка заключается в  вычитании константы 0.5  из каждого модуля разности наблюденного $O_{i}$ и ожидаемого $E_{i}$ значений, то есть, статистика с такой поправкой выглядит так:

$χ_{Yates}^2=\sum_{i=1}^N\frac{\left(|O_{i}-E_{i}|-0.5\right)^{2}}{E_{i}}$

Такая поправка, как несложно догадаться по формуле, как правило, уменьшает  значение статистики $χ^{2}$, то есть увеличивает достигаемый уровень значимости.

Эта поправка обычно используется для таблиц сопряженности размером 2x2  и для небольшого количества наблюдений. Такая поправка, однако, не является серебрянной пулей, и часто критикуется за то, что статистический критерий при ее использовании становится слишком консервативным, то есть часто не отвергает нулевую гипотезу там, где она неверна (совершает ошибку II рода). 

Полезно знать, что эта поправка часто включена по  умолчанию (например, в функции scipy.stats.chi2_contingency) и понимать ее влияние на оценку достигаемого уровня значимости. 

Проведите те же самые сравнения, что и в вопросе №1, только с включенной коррекцией:

scipy.stats.chi2_contingency(subtable, correction=True)

In [10]:
chi2_pvalue_lower_0_05_count = 0
for contingency_table in contingency_table_list:
    if stats.chi2_contingency(contingency_table, correction=True)[1] < 0.05:
        chi2_pvalue_lower_0_05_count += 1
print 'Количество пар, в которых p-value меньше 0.05 с учетом поправки: ', chi2_pvalue_lower_0_05_count

Количество пар, в которых p-value меньше 0.05 с учетом поправки:  0


Что если у нас мало данных,  мы не хотим использовать аппроксимацию дискретного распределения непрерывным и использовать сомнительную поправку,  предположения  критерия xи-квадрат не выполняются, а проверить гипотезу о том, что данные принадлежат одному распределению, нужно?

В таком случае прибегают к так называемому точному критерию Фишера. Этот критерий не использует приближений  и в точности вычисляет значение достигаемого уровня значимости используя комбинаторный подход.

Пусть у нас есть таблица сопряженности 2x2:

|                   ||Группа 1|Группа 2|        $Σ$|
|--------------------|--------|--------|-----------|
|<b>Воздействие 1</b>|     $a$|     $b$|      $a+b$|
|<b>Воздействие 2</b>|     $c$|     $d$|      $c+d$|
|                 $Σ$|   $a+c$|   $b+d$|$n=a+b+c+d$|

Тогда вероятность получить именно такие $a, b, c, d$ при фиксированных значениях сумм по строкам и по столбцам) задается выражением:

$p=\frac{\left(\begin{array}{c}{a+b}\\ a\end{array}\right)\left(\begin{array}{c}{c+d}\\ c\end{array}\right)}{\left(\begin{array}{c}n\\ {a+c}\end{array}\right)}=\frac{(a+b)!(c+d)!(a+c)!(b+d)!}{a!b!c!d!n!}$.

В числителе этой дроби стоит суммарное количество способов выбрать $a$ и $c$ из $a+b$ и $c+d$ соответственно. А в знаменателе — количество способов выбрать число объектов, равное сумме элементов первого столбца $a+c$ из общего количества рассматриваемых объектов $n$.

Чтобы посчитать достигаемый уровень значимости критерия Фишера, нужно перебрать все возможные значения $a,b,c,d,$ в клетках этой таблицы так, чтобы построковые и постолбцовые суммы не изменились. Для каждого такого набора $a,b,c,d$ нужно вычислить значение $p_{i}$ по формуле выше и просуммировать все такие значения $p_{i}$, которые меньше или равны $p$, которое мы вычислили по наблюдаемым значениям $a,b,c,d$.

Понятно, что такой критерий вычислительно неудобен в силу большого количества факториалов в формуле выше. То есть даже при небольших выборках для вычисления значения этого критерия приходится оперировать очень большими числами. Поэтому данным критерием пользуются обычно только для таблиц 2x2, но сам критерий никак не ограничен количеством строк и столбцов, и его можно построить для любой таблицы $n×m$.

Посчитайте для каждой пары штатов, как и в первом задании, достигаемый уровень значимости с помощью точного критерия Фишера и сравните получившиеся значения с двумя другими подходами, описанными выше.

Точный критерий Фишера удобно вычислять с помощью функции 

scipy.stats.fisher_exact

которая принимает на вход таблицу сопряженности 2x2. 

In [11]:
chi2_pvalue_lower_0_05_count = 0
for contingency_table in contingency_table_list:
    if stats.fisher_exact(contingency_table)[1] < 0.05:
        chi2_pvalue_lower_0_05_count += 1
print 'Количество пар, в которых p-value меньше 0.05 по критерию Фишера: ', chi2_pvalue_lower_0_05_count

Количество пар, в которых p-value меньше 0.05 по критерию Фишера:  10


Давайте попробуем применить полученные знания о разных видах корреляции и ее применимости на практике. 

Рассмотрим пару признаков <b>day_calls</b> и  <b>mes_estim</b>. Посчитайте корреляцию Пирсона между этими признаками на всех данных, ее значимость.

In [12]:
corr_calc_res = stats.pearsonr(df['day_calls'], df['mes_estim'])
print 'Корреляция Пирсона: ', corr_calc_res[0]
print 'p-value: ', corr_calc_res[1]

Корреляция Пирсона:  -0.051794350587572625
p-value:  0.0027798836869756707


Еще раз рассмотрим пару признаков day_calls и  mes_estim. Посчитайте корреляцию Спирмена между этими признаками на всех данных, ее значимость.

In [13]:
corr_calc_res = stats.spearmanr(df['day_calls'], df['mes_estim'])
print 'Корреляция Спирмена: ', corr_calc_res[0]
print 'p-value: ', corr_calc_res[1]

Корреляция Спирмена:  0.043349880533927444
p-value:  0.012317367189170541


Посчитайте значение коэффицента корреляции Крамера между двумя признаками:  штатом (state) и оттоком пользователей (churn) для всех пользователей, которые находились в контрольной группе (treatment=1).  Что можно сказать о достигаемом уровне значимости при проверке гипотезы о равенство нулю этого коэффициента?

In [14]:
state_samp_num_more_40_num = 0
for state_samp_num in states_treatment_1_churn_counts_pd.sum(axis=1):
    if state_samp_num > 40:
        state_samp_num_more_40_num += 1
print 'Штаты с выборкой больше, чем 40: ', state_samp_num_more_40_num

Штаты с выборкой больше, чем 40:  0


Так как все выборки для всех штатов меньше 40, то нет смысла считать коэффициент V Крамера, потому что в нем используется статистика хи-квадрат, которую можно вычислять при выборке больше 40.

Вы прослушали большой курс и к текущему моменту обладете достаточными знаниями, чтобы попытаться самостоятельно выбрать нужный метод / инструмент / статистический критерий и сделать правильное заключение. 

В этой части задания вам нужно будет самостоятельно решить, с помощью каких методов можно провести анализ эффективности удержания (churn) с помощью раличных методов (treatment = 0, treatment = 2) относительно контрольной группы пользователей (treatment = 1). 

Что можно сказать об этих двух методах (treatment = 0, treatment = 2)? Одинаковы ли они с точки зрения эффективности? Каким бы методом вы бы посоветовали воспользоваться компании?

<i>Не забудьте про поправку на множественную проверку!  И не пользуйтесь односторонними альтернативами, поскольку вы не знаете, к каким действительно последствиям приводят тестируемые методы  (treatment = 0, treatment = 2) !</i>

In [15]:
df_treatment_0 = df[df['treatment'] == 0]
df_treatment_0['churn'][df_treatment_0['churn'] == 'False.'] = 0
df_treatment_0['churn'][df_treatment_0['churn'] == 'True.'] = 1
df_treatment_2 = df[df['treatment'] == 2]
df_treatment_2['churn'][df_treatment_2['churn'] == 'False.'] = 0
df_treatment_2['churn'][df_treatment_2['churn'] == 'True.'] = 1

/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Volumes/Storage/Applications/Programming/anaconda3/envs/py2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.p

In [16]:
treatment_0_churn_0 = len(df_treatment_0[df_treatment_0['churn'] == 0])
treatment_0_churn_1 = len(df_treatment_0[df_treatment_0['churn'] == 1])
treatment_1_churn_0 = len(df_treatment_1[df_treatment_1['churn'] == 0])
treatment_1_churn_1 = len(df_treatment_1[df_treatment_1['churn'] == 1])
treatment_2_churn_0 = len(df_treatment_2[df_treatment_2['churn'] == 0])
treatment_2_churn_1 = len(df_treatment_2[df_treatment_2['churn'] == 1])

In [17]:
contingency_table = np.array([[treatment_0_churn_0, treatment_0_churn_1],
                              [treatment_1_churn_0, treatment_1_churn_1],
                              [treatment_2_churn_0, treatment_2_churn_1]])
pd.DataFrame(contingency_table, index=['treatment_0', 'treatment_1', 'treatment_2'])

,0,1
treatment_0,968,165
treatment_1,917,180
treatment_2,965,138


Проверяются две нулевые гипотезы $H_{00}$ о равенстве доли удержанных абонентов в группах с методом удержания 0 (treatment_0) и контрольной группе (tretment_1) и $H_{02}$ о равенстве доли удержанных абонентов в группах с методом удержания 2 (treatment_2) и контрольной группе (tretment_1) против двухсторонних альтернатив $H_{10}$ и $H_{12}$.

Обе гипотезы будут проверятся двухстороннем Z-критерием для разности долей для независимой выборки с поправкой на множественную провеку гипотез методом Холма.

In [18]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [19]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [20]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [21]:
p_val_0 = proportions_diff_z_test(proportions_diff_z_stat_ind(df_treatment_0['churn'], df_treatment_1['churn']))
p_val_2 = proportions_diff_z_test(proportions_diff_z_stat_ind(df_treatment_2['churn'], df_treatment_1['churn']))

In [22]:
reject, p_corrected, a1, a2 = multipletests([p_val_0, p_val_2], alpha = 0.05, method = 'holm')

In [23]:
print 'p-value проверки H00: ', p_corrected[0], '\n', 'p-value проверки H02: ', p_corrected[1]

p-value проверки H00:  0.2283311639045107 
p-value проверки H02:  0.018696168588902218


Согласно результатам проверок нулевая гипотеза $H_{00}$ не отвергается на уровне значимости 95%, то есть метод удержания 0 (tratment_0) не эффективен.

Нулевая гипотеза $H_{02}$ отвергается на уровне значимости 95% в пользу двухсторонней альтернативы $H_{12}$, то есть метод удержания 2 (tratment_2) эффективен.